## ipyleaflet and mapquest

Get your free API key here: https://developer.mapquest.com/

The API documentation is available here:
https://developer.mapquest.com/documentation/open/directions-api/

The following wrapper is suggested. Feel free to adapt!

In [ ]:
from functools import lru_cache

import requests
from ipyleaflet import Map, Marker, Polyline
from typing import Any, Dict, List

JSONType = Dict[str, Any]


class Point:
    def __init__(self, json: JSONType) -> None:
        self.json = json
        self.latlng = json["results"][0]["locations"][0]["latLng"]

    def __repr__(self) -> str:
        x = self.json["results"][0]["locations"][0]
        return ", ".join(
            x[f"adminArea{i}"]
            for i in reversed(range(1, 7))
            if f"adminArea{i}" in x and x[f"adminArea{i}"] != ""
        )

    def add_to(self, m: Map, **kwargs) -> None:
        m.add_layer(
            Marker(location=[self.latlng["lat"], self.latlng["lng"]], **kwargs)
        )


class Route:
    def __init__(self, json: JSONType) -> None:
        self.json = json

    @property
    def distance(self) -> float:
        return self.json["route"]["distance"]

    # lru_cache provides the most basic form of cache
    # think of better strategies if you need to save requests
    @lru_cache(None)
    def get_shape(self) -> List[float]:
        return MapQuest.routeshape(self.json["route"]["sessionId"])

    def add_to(self, m: Map, **kwargs) -> None:
        kwargs = {"fill_opacity": 0, **kwargs}
        it = iter(self.get_shape())

        def getTuple(it):
            while True:
                try:
                    yield next(it), next(it)
                except StopIteration:
                    return

        m.add_layer(Polyline(locations=list(getTuple(it)), **kwargs))


class MapQuest:

    # fill here your API key
    api_key = ""

    @staticmethod
    def geolocate(name: str) -> Point:
        base_url = "http://www.mapquestapi.com/geocoding/v1/address"

        req = requests.post(
            base_url, params={"key": MapQuest.api_key}, json={"location": name}
        )

        return Point(req.json())

    @staticmethod
    def route(from_: str, to_: str) -> Route:
        base_url = "http://www.mapquestapi.com/directions/v2/route"

        req = requests.post(
            base_url,
            params={"key": MapQuest.api_key},
            json={"locations": [from_, to_]},
        )

        return Route(req.json())

    @staticmethod
    def routeshape(sessionId: str) -> List[float]:
        base_url = "http://open.mapquestapi.com/directions/v2/routeshape"

        req = requests.get(
            base_url,
            params={
                "key": MapQuest.api_key,
                "sessionId": sessionId,
                "fullShape": True,
            },
        )
        json = req.json()
        return json["route"]["shape"]["shapePoints"]


In [ ]:
m = Map(
    center=(44, 2), zoom=7,
)
m

In [ ]:
MapQuest.geolocate('Toulouse').add_to(m)
MapQuest.geolocate('Bordeaux').add_to(m)

route = MapQuest.route('Toulouse', 'Perpignan')
route.add_to(m)
route.distance

In [ ]:
MapQuest.route('Montpellier', 'Rodez').add_to(m, color='red')